Imports

In [ ]:
from time import time
import logging
import matplotlib.pyplot as plt
#
import random, os
import cv2
import numpy as np
import json
import math


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import learning_curve

Face and eyes detection functions

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image,minNeighbors=6,minSize=(75, 75)) #parameters change depending the dataset
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        
        return False, "zero"
    else: #if more than 1 faces are detected, the biggest is kept (the smaller ones usually belong in the background)
        biggest=0
        pos=0
        for i,(x,y,w,h) in enumerate(face):
            if biggest<(h*w):
                biggest=h*w
                pos=i
        return True, image[y:y+w, x:x+h], face[pos]

In [ ]:
def face_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_frontalface_default.xml')
    face = haar_classifier.detectMultiScale(image)#,scaleFactor=1.2, minNeighbors=5)
    if (len(face)==1): #if 1 face is found
        (x,y,w,h) = face[0]
        '''
        for (x,y,w,h) in face: 
            cv2.rectangle(image, (x,y), (x+w,y+h), (255,255,255), 3)
        plt.imshow(image)
        plt.show()
        '''
        return True, image[y:y+w, x:x+h], face[0]
    elif (len(face)==0): #if no faces are detected
        return False, "zero"
    else: #if more than 1 faces are detected
        return False, "more"

In [ ]:
def eye_detection(image):
    #image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image=cv2.resize(image, (200, 200), interpolation=cv2.INTER_LINEAR)
    haar_classifier = cv2.CascadeClassifier('C:/Users/Thomas/Documents/face_detection/haarcascades/haarcascade_eye.xml')
    eyes = haar_classifier.detectMultiScale(image, minNeighbors=4)#, minSize=(30, 30))
    if (len(eyes)==2): #if 2 eyes are detected
        #(x,y,w,h) = face[0]
        return True, eyes
    else:      #if the number of eyes found is different from 2
        return False, "more or less"

In [ ]:
datapath=('C:/Users/Thomas/Documents/Datasets/lfw(250x250)/')
min_faces=30
faces=[]
labels=[]

In [ ]:
#some counters
photos_with_face=0
photos_with_0_faces=0
photos_with_more_faces=0

In [ ]:
#some more counters
photos_with_2_eyes=0
photos_with__eyes=0

Saving the data/images and preprocessing

In [ ]:
target_names=[]  #all the names are saved here
total_photos_seen=0 
n_classes=0
folders = os.listdir(datapath)
for folder in folders:
    label = os.path.basename(folder)
    training_images_path = datapath + '/' + folder
    num_of_faces = len(os.listdir(training_images_path))
    #if num_of_faces>=min_faces:   #people with low number of faces are skipped
    target_names.append(label)
    n_classes=n_classes+1
    faces_per_person=0
    for image in os.listdir(training_images_path):
        total_photos_seen=total_photos_seen+1
        image_path = training_images_path + '/' + image
        training_image = cv2.imread(image_path)
        #face=cv2.cvtColor(training_image,cv2.COLOR_BGR2GRAY)
        #resized=cv2.resize(face,(224,224),interpolation=cv2.INTER_AREA)
        result_im=face_detection(training_image) #this function returns TRUE/FALSE, the cut image and the coordinates of where the cut was made
        if result_im[0]: #if TRUE (face detected) then the cut face goes into the array
            cut_face=result_im[1]
            faces.append(cut_face)
            labels.append(n_classes)

        '''
        if faces_per_person==45:
                break
        '''        

In [ ]:
new_faces=[]
new_labels=[]

In [ ]:
rotated_faces=0
#unwanted_images=[3,110,113,166,238,245,349,398,444,903,978,1194,1218] 
for i in range(len(faces)):
    #if i not in unwanted_images: in case there are pictures that cannot be processed
    training_image = cv2.imread(faces[i])
    result_im=eye_detection(training_image) #finding the place of the eyes
    
    if result_im[0]: #process of finding the angle of the two eyes, and alighnment
        rotated_faces=rotated_faces+1
        (x1,y1,w1,h1)=result_im[1][0]
        (x2,y2,w2,h2)=result_im[1][1]
        eye1_center_x=x1+(w1/2)
        eye1_center_y=y1+(h1/2)
        eye2_center_x=x2+(w2/2)
        eye2_center_y=y2+(h2/2)
        if (eye1_center_x>eye2_center_y):
            deltaY = eye1_center_y - eye2_center_y 
            deltaX = eye1_center_x - eye2_center_x
        else:
            deltaY = eye2_center_y - eye1_center_y 
            deltaX = eye2_center_x - eye1_center_x
            
        my_radians = math.atan2(deltaY , deltaX)
        my_degrees = math.degrees(my_radians)

        #rotation
        rows, cols = training_image.shape[:2]
        M = cv2.getRotationMatrix2D((cols/2, rows/2), my_degrees, 1)
        img_rotated = cv2.warpAffine(training_image, M, (cols,rows))
        '''
        font = cv2.FONT_HERSHEY_SIMPLEX
        plt.text(0,0,i)
        plt.imshow(img_rotated)
        plt.show()
        '''

        #scaling (all images will become 200x200)
        #if before the conversion the image is smaller than 200x200 INTER_LINEAR will be used, else INTER_AREA will be used
        image_size=rows*cols        
        if (image_size<=40000):
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_LINEAR)
        else:
            scaled_image=cv2.resize(img_rotated, (200, 200), interpolation=cv2.INTER_AREA)

        #normalization
        norm_img = np.zeros((200, 200))
        norm_img = cv2.normalize(scaled_image, norm_img, 0, 255, cv2.NORM_MINMAX)

        final_image = cv2.cvtColor(norm_img, cv2.COLOR_BGR2GRAY)

        new_faces.append(final_image)
        new_labels.append(labels[i])

faces=new_faces
labels=new_labels

Finding the components parameter

In [ ]:
#"flattening" the faces for the PCA
flat_faces = []

for face in faces:
    flat_faces.append(face.reshape(-1))

flat_faces = np.array(flat_faces)
labels=np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(flat_faces, labels, test_size=0.1, random_state=42)
X_train = np.array(X_train)
X_test=np.array(X_test)

In [ ]:
#Firstly, PCA is fitted without the component parameter, so we can find the optimal number of components
t0=time()
pca = PCA().fit(X_train)

plt.figure(figsize=(18, 7))
plt.plot(pca.explained_variance_ratio_.cumsum(), lw=3)
print("done in %0.3fs" % (time() - t0))

In [ ]:
#the smallest value is the optimal number of components
np.where(pca.explained_variance_ratio_.cumsum() > 0.95)

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import WhiteKernel

In [ ]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle


In [ ]:
#with open('eigen.p', 'rb') as fp:
#    data = pickle.load(fp)
#open a pickle file

In [ ]:
#number of components
#small dataset 42
#bioID 139 (preprocessed 140)
#facescrub 289(250x250),312(500x500),301(350x350), 270(200x200) 251 (preprocessed)
#lfw 374 (preprocessed 210)
#lfwcropped  158
#lfw2 (preprocessed 210)
#extended 142

In [ ]:
pca = PCA(n_components=142, svd_solver='randomized',whiten=True)
lda = LDA()
cv=StratifiedKFold(n_splits=10)

In [ ]:
scoring = {'accuracy': 'accuracy',
           'precision_macro': 'precision_macro',
           'precision_micro': 'precision_micro',
           'precision_weighted': 'precision_weighted',
           'recall_macro': 'recall_macro',
           'recall_micro': 'recall_micro',
           'recall_weighted': 'recall_weighted',           
           'f1_macro':'f1_macro',
           'f1_micro':'f1_micro',
           'f1_weighted':'f1_weighted'}

In [ ]:
#the dictionaries will be filled with the results, and then the dictionaries will be saved in a pickle file

In [ ]:
results_eigen = dict({'svc':[],'mlp':[],'knn':[],'logreg':[],'linsvc':[],'tree':[],'forrest':[],'adaboost':[],'gnb':[],'gpc':[],'ridge':[]})

In [ ]:
results_fisher = dict({'svc':[],'mlp':[],'knn':[],'logreg':[],'linsvc':[],'tree':[],'forrest':[],'adaboost':[],'gnb':[],'gpc':[],'ridge':[]})

In [ ]:
eigenfaces_learning_curve=dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,
                                'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})

In [ ]:
fisherfaces_learning_curve=dict({'svc':None,'mlp':None,'knn':None,'logreg':None,'linsvc':None,
                                'tree':None,'forrest':None,'adaboost':None,'gnb':None,'gpc':None,'ridge':None})

Eigenfaces

svc

In [ ]:
clf=SVC(class_weight='balanced',C=28.104130383007977,gamma=7.281641758356972,kernel='poly',degree=1)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('clf', clf)])

In [ ]:
t0 = time()
svm_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['svc']=svm_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
svc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['svc']=svc_learning_curve

KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3,p=2,leaf_size=50)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('knn', knn)])

In [ ]:
t0 = time()
knn_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['knn']=knn_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
knn_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['knn']=knn_learning_curve

mlp

In [ ]:
mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(333,200,496,347,495),alpha=0.08193742846113258,
                    learning_rate='invscaling', activation='tanh',momentum=0.5725450935194276,verbose=False, early_stopping=True)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('mlp', mlp)])

In [ ]:
t0 = time()
mlp_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['mlp']=mlp_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
mlp_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['mlp']=mlp_learning_curve

logreg

In [ ]:
logreg = LogisticRegression(class_weight='balanced',C=33.60545334855496,solver='sag')

In [ ]:
pipe = Pipeline([('pca', pca),
                ('logreg', logreg)])

In [ ]:
t0 = time()
logreg_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['logreg']=logreg_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
logreg_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['logreg']=logreg_learning_curve

linearsvm

In [ ]:
linear_svm = LinearSVC(C=95.48139914999754,loss='hinge')

In [ ]:
pipe = Pipeline([('pca', pca),
                ('linear_svm', linear_svm)])

In [ ]:
t0 = time()
linsvc_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['linsvc']=linsvc_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
linsvc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['linsvc']=linsvc_learning_curve

decision trees

In [ ]:
tree = DecisionTreeClassifier(random_state=0,min_samples_leaf=1,max_depth=15,min_impurity_decrease=0.021318938975762092,criterion='entropy',max_features=10,
                              max_leaf_nodes=None)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('tree', tree)])

In [ ]:
t0 = time()
tree_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['tree']=tree_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
tree_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['tree']=tree_learning_curve

random forrest

In [ ]:
forrest = RandomForestClassifier(random_state=0,min_samples_leaf=3,max_depth=10,n_estimators=387,class_weight='balanced',bootstrap=True)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('forrest', forrest)])

In [ ]:
t0 = time()
forrest_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['forrest']=forrest_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
forrest_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['forrest']=forrest_learning_curve

adaboost

In [ ]:
adaboost = AdaBoostClassifier(n_estimators=264,random_state=0,learning_rate=0.2131171118825734)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('adaboost', adaboost)])

In [ ]:
t0 = time()
ada_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['adaboost']=ada_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
adaboost_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['adaboost']=adaboost_learning_curve

gaussianNB

In [ ]:
gnb = GaussianNB()

In [ ]:
pipe = Pipeline([('pca', pca),
                ('gnb', gnb)])

In [ ]:
t0 = time()
gnb_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['gnb']=gnb_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
gnb_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['gnb']=gnb_learning_curve

gaussian process

In [ ]:
gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=0.840398547098756),random_state=0)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('gpc', gpc)])

In [ ]:
t0 = time()
gpc_score=cross_validate(pipe, flat_faces, labels, cv=cv_for_gpc,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['gpc']=gpc_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv_for_gpc,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
gpc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['gpc']=gpc_learning_curve

ridge

In [ ]:
ridge = RidgeClassifier()

In [ ]:
pipe = Pipeline([('pca', pca),
                ('ridge', ridge)])

In [ ]:
t0 = time()
ridge_score=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_eigen['ridge']=ridge_score

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
ridge_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
eigenfaces_learning_curve['ridge']=ridge_learning_curve

Saving data in pickle files

In [ ]:
with open('eigenfaces_extyale.p', 'wb') as fp:
    pickle.dump(results_eigen, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('eigenfaces_learning_curve_extyale.p', 'wb') as fp:
    pickle.dump(eigenfaces_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)

fisherfaces

svc

In [ ]:
svc=SVC(kernel='linear', class_weight='balanced',C=3.9241808801638385,gamma=96.58400875342846,degree=6)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('svc', svc)])

In [ ]:
t0 = time()
svm_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['svc']=svm_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
svc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['svc']=svc_learning_curve

knn

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13,p=1,leaf_size=32)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('knn', knn)])

In [ ]:
t0 = time()
knn_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['knn']=knn_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
knn_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['knn']=knn_learning_curve

mlp

In [ ]:
mlp = MLPClassifier(solver='adam',hidden_layer_sizes=(417,469,290,194),alpha=0.011508598976828089,
                    learning_rate='constant', activation='tanh',momentum=0.5812137485951651,verbose=False, early_stopping=True)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('mlp', mlp)])

In [ ]:
t0 = time()
mlp_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['mlp']=mlp_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
mlp_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['mlp']=mlp_learning_curve

logreg

In [ ]:
logreg = LogisticRegression(C=26.23190488581573,solver='lbfgs',class_weight='balanced')

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('logreg', logreg)])

In [ ]:
t0 = time()
logreg_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['logreg']=logreg_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
logreg_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['logreg']=logreg_learning_curve

linearsvc

In [ ]:
linear_svm = LinearSVC(C=0.851388416540332,loss='hinge',class_weight='balanced',fit_intercept=False)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('linear_svm', linear_svm)])

In [ ]:
t0 = time()
linsvc_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['linsvc']=linsvc_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
linsvc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['linsvc']=linsvc_learning_curve

decision tree 

In [ ]:
tree = DecisionTreeClassifier(random_state=0,criterion='gini',min_impurity_decrease=0.03861521502801396,max_depth=19,min_samples_leaf=2,max_leaf_nodes=None,max_features=8)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda',lda),
                ('tree', tree)])

In [ ]:
t0 = time()
tree_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['tree']=tree_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
tree_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['tree']=tree_learning_curve

forrest

In [ ]:
forrest = RandomForestClassifier(random_state=0,max_depth=9,min_samples_leaf=3,n_estimators=241,class_weight='balanced',bootstrap=True)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('forrest', forrest)])

In [ ]:
t0 = time()
forrest_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['forrest']=forrest_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
forrest_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['forrest']=forrest_learning_curve

adaboost

In [ ]:
adaboost = AdaBoostClassifier(n_estimators=174,random_state=0,learning_rate=0.28183708257574086)

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('adaboost', adaboost)])

In [ ]:
t0 = time()
ada_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['adaboost']=ada_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
adaboost_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['adaboost']=adaboost_learning_curve

gaussiannb

In [ ]:
gnb = GaussianNB()

In [ ]:
pipe = Pipeline([('pca', pca),
                 ('lda', lda),
                ('gnb', gnb)])

In [ ]:
t0 = time()
gnb_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['gnb']=gnb_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
gnb_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['gnb']=gnb_learning_curve

gaussian process

In [ ]:
gpc = GaussianProcessClassifier(kernel=1.0*RBF(length_scale=1.0687440294594166),random_state=0)

In [ ]:
pipe=Pipeline([('pca', pca),
                ('lda',lda),
                ('gpc', gpc)])

In [ ]:
t0 = time()
gpc_score_=cross_validate(pipe, flat_faces, labels, cv=cv_for_gpc,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['gpc']=gpc_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv_for_gpc,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
gpc_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['gpc']=gpc_learning_curve

ridge

In [ ]:
ridge = RidgeClassifier(alpha=0.5)

In [ ]:
pipe = Pipeline([('pca', pca),
                ('lda',lda),
                ('ridge', ridge)])

In [ ]:
t0 = time()
ridge_score_=cross_validate(pipe, flat_faces, labels, cv=cv,scoring=scoring,return_train_score=True,n_jobs=-1)
print("done in %0.3fs" % (time() - t0))

In [ ]:
results_fisher['ridge']=ridge_score_

In [ ]:
t0 = time()
train_sizes, train_scores, test_scores, fit_times, score_times =learning_curve(pipe, flat_faces, labels, n_jobs=4,
                       cv=cv,train_sizes=np.linspace(0.1, 1.0, 5),
                       return_times=True)
print("done in %0.3fs" % (time() - t0))
ridge_learning_curve=dict({1:train_sizes,2:train_scores,3:test_scores,4:fit_times,5:score_times})
fisherfaces_learning_curve['ridge']=ridge_learning_curve

Saving data in pickle files

In [ ]:
with open('fisher_extyale.p', 'wb') as fp:
    pickle.dump(results_fisher, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('fisher_learning_curve_extyale.p', 'wb') as fp:
    pickle.dump(fisherfaces_learning_curve, fp, protocol=pickle.HIGHEST_PROTOCOL)